In [1]:
# Load Dataset
import tensorflow as tf

# Balance
train_dir = '/kaggle/input/tomatodiseasedataset/TomatoDiseaseSplit/train'
val_dir = '/kaggle/input/tomatodiseasedataset/TomatoDiseaseSplit/val'
test_dir = '/kaggle/input/tomatodiseasedataset/TomatoDiseaseSplit/test'

# Define parameters
BATCH_SIZE = 32
IMG_SIZE = (256, 256)  # Image dimensions

# Load the train dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                    shuffle=True,
                                                                    batch_size=BATCH_SIZE,
                                                                    image_size=IMG_SIZE)


# Load the validation dataset
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(val_dir,
                                                                  shuffle=True,
                                                                  batch_size=BATCH_SIZE,
                                                                  image_size=IMG_SIZE)


# Load the test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                   shuffle=False,
                                                                   batch_size=BATCH_SIZE,
                                                                   image_size=IMG_SIZE)

Found 9973 files belonging to 10 classes.
Found 1595 files belonging to 10 classes.
Found 1607 files belonging to 10 classes.


In [2]:
# Print class names
class_names = train_dataset.class_names
print("Class names:", class_names)

Class names: ['Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


In [3]:
# Define normalization layer
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Apply normalization to each dataset
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

In [4]:
# Apply prefetching
AUTOTUNE = tf.data.AUTOTUNE  # Automatically determine the optimal buffer size

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [5]:
# Define input shape, number of classes and batch size
input_shape = (256, 256, 3)
num_classes = 10  
batch_size = 32

# Custom CNN

## Trial 1: 3 Block

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models,optimizers

# Function to build the Tomato Disease CNN
def build_tomato_disease_cnn(input_shape, num_classes):
    model = models.Sequential([
        # Input Layer
        layers.Input(shape=input_shape),
        
        # First Convolutional Layer
        layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Second Convolutional Layer
        layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Third Convolutional Layer
        layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        # Flatten the Output
        layers.Flatten(),
        
        # Dropout Layer (50%)
        layers.Dropout(0.5),
        
        # Fully Connected Dense Layer
        layers.Dense(units=128, activation='relu'),
        
        # Dropout Layer (30%)
        layers.Dropout(0.3),
        
        # Output Layer
        layers.Dense(units=num_classes, activation='softmax')
    ])
    return model

input_shape = (256, 256, 3)  # Adjust according to your dataset (image height, width, channels)
num_classes = 10             # Replace with the actual number of classes in your dataset

# Build the model
cnn_model = build_tomato_disease_cnn(input_shape, num_classes)

In [7]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Compile the model with learning rate 0.0001
optimizer = optimizers.Adam(learning_rate=0.0001)
cnn_model.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model
history = cnn_model.fit(train_dataset, 
                          validation_data=val_dataset, 
                          epochs=30, 
                          batch_size=batch_size,
                          callbacks=[early_stopping, reduce_lr])

Epoch 1/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 34s 80ms/step - accuracy: 0.3053 - loss: 1.9124 - val_accuracy: 0.7122 - val_loss: 0.9377 - learning_rate: 1.0000e-04
Epoch 2/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 25s 53ms/step - accuracy: 0.7025 - loss: 0.8858 - val_accuracy: 0.7517 - val_loss: 0.7226 - learning_rate: 1.0000e-04
Epoch 3/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 16s 53ms/step - accuracy: 0.7731 - loss: 0.6730 - val_accuracy: 0.8238 - val_loss: 0.5240 - learning_rate: 1.0000e-04
Epoch 4/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 17s 54ms/step - accuracy: 0.8179 - loss: 0.5518 - val_accuracy: 0.8445 - val_loss: 0.4444 - learning_rate: 1.0000e-04
Epoch 5/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 17s 55ms/step - accuracy: 0.8472 - loss: 0.4691 - val_accuracy: 0.8671 - val_loss: 0.3899 - learning_rate: 1.0000e-04
Epoch 6/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 17s 56ms/step - accuracy: 0.8678 - loss: 0.4056 - val_accuracy: 0.8746 - val_loss: 0.3657 - learning_rate: 1.0000e-04
Epoch 7/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 18s 57ms/ste

In [8]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = cnn_model.evaluate(test_dataset)
print(f'Test accuracy: {test_accuracy}')
print(f'Test loss: {test_loss}')

51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - accuracy: 0.9257 - loss: 0.2614
Test accuracy: 0.936527669429779
Test loss: 0.2195313274860382


## 

## Trial 2: 4 Block

In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Function to build the Tomato Disease CNN
def build_tomato_disease_cnn(input_shape, num_classes):
    model = models.Sequential([
        # Input Layer
        layers.Input(shape=input_shape),
        
        # First Convolutional Layer
        layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Second Convolutional Layer
        layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Third Convolutional Layer
        layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Fourth Convolutional Layer
        layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Flatten the Output
        layers.Flatten(),
        
        # Dropout Layer (50%)
        layers.Dropout(0.5),
        
        # Fully Connected Dense Layer
        layers.Dense(units=128, activation='relu'),
        
        # Dropout Layer (30%)
        layers.Dropout(0.3),
        
        # Output Layer
        layers.Dense(units=num_classes, activation='softmax')
    ])
    return model

# Example Usage
# Specify the input shape and number of classes
input_shape = (256, 256, 3)  # Adjust according to your dataset (image height, width, channels)
num_classes = 10             # Replace with the actual number of classes in your dataset

# Build the model
cnn_model_2 = build_tomato_disease_cnn(input_shape, num_classes)

In [10]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Compile the model with learning rate 0.0001
optimizer = optimizers.Adam(learning_rate=0.0001)
cnn_model_2.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model
history = cnn_model_2.fit(train_dataset, 
                          validation_data=val_dataset, 
                          epochs=30, 
                          batch_size=batch_size,
                          callbacks=[early_stopping, reduce_lr])

Epoch 1/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 26s 70ms/step - accuracy: 0.2860 - loss: 1.9569 - val_accuracy: 0.7292 - val_loss: 0.8465 - learning_rate: 1.0000e-04
Epoch 2/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 18s 58ms/step - accuracy: 0.6832 - loss: 0.9176 - val_accuracy: 0.8038 - val_loss: 0.5859 - learning_rate: 1.0000e-04
Epoch 3/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 18s 58ms/step - accuracy: 0.7853 - loss: 0.6329 - val_accuracy: 0.8263 - val_loss: 0.5074 - learning_rate: 1.0000e-04
Epoch 4/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 18s 58ms/step - accuracy: 0.8247 - loss: 0.5302 - val_accuracy: 0.8596 - val_loss: 0.4091 - learning_rate: 1.0000e-04
Epoch 5/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 18s 58ms/step - accuracy: 0.8477 - loss: 0.4551 - val_accuracy: 0.8846 - val_loss: 0.3472 - learning_rate: 1.0000e-04
Epoch 6/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 18s 58ms/step - accuracy: 0.8706 - loss: 0.3966 - val_accuracy: 0.8727 - val_loss: 0.3526 - learning_rate: 1.0000e-04
Epoch 7/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/ste

In [11]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = cnn_model_2.evaluate(test_dataset)
print(f'Test accuracy: {test_accuracy}')
print(f'Test loss: {test_loss}')

51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.9440 - loss: 0.1856
Test accuracy: 0.9502177834510803
Test loss: 0.1624944806098938


## Trial 3: 5 Block

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Function to build the Tomato Disease CNN
def build_tomato_disease_cnn(input_shape, num_classes):
    model = models.Sequential([
        # Input Layer
        layers.Input(shape=input_shape),
        
        # First Convolutional Layer
        layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Second Convolutional Layer
        layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Third Convolutional Layer
        layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Fourth Convolutional Layer
        layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Fifth Convolutional Layer
        layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Flatten the Output
        layers.Flatten(),
        
        # Dropout Layer (50%)
        layers.Dropout(0.5),
        
        # Fully Connected Dense Layer
        layers.Dense(units=128, activation='relu'),
        
        # Dropout Layer (30%)
        layers.Dropout(0.3),
        
        # Output Layer
        layers.Dense(units=num_classes, activation='softmax')
    ])
    return model

input_shape = (256, 256, 3)  # Adjust according to your dataset (image height, width, channels)
num_classes = 10             # Replace with the actual number of classes in your dataset

# Build the model
cnn_model_3 = build_tomato_disease_cnn(input_shape, num_classes)

In [13]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Compile the model with learning rate 0.0001
optimizer = optimizers.Adam(learning_rate=0.0001)
cnn_model_3.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model
history = cnn_model_3.fit(train_dataset, 
                          validation_data=val_dataset, 
                          epochs=30, 
                          batch_size=batch_size,
                          callbacks=[early_stopping, reduce_lr])

Epoch 1/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 26s 70ms/step - accuracy: 0.1698 - loss: 2.1671 - val_accuracy: 0.5912 - val_loss: 1.3042 - learning_rate: 1.0000e-04
Epoch 2/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 19s 59ms/step - accuracy: 0.5766 - loss: 1.2267 - val_accuracy: 0.7436 - val_loss: 0.7954 - learning_rate: 1.0000e-04
Epoch 3/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - accuracy: 0.7085 - loss: 0.8515 - val_accuracy: 0.8038 - val_loss: 0.5969 - learning_rate: 1.0000e-04
Epoch 4/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 18s 58ms/step - accuracy: 0.7562 - loss: 0.6970 - val_accuracy: 0.8213 - val_loss: 0.5376 - learning_rate: 1.0000e-04
Epoch 5/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 18s 58ms/step - accuracy: 0.7782 - loss: 0.6196 - val_accuracy: 0.8470 - val_loss: 0.4615 - learning_rate: 1.0000e-04
Epoch 6/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - accuracy: 0.8151 - loss: 0.5379 - val_accuracy: 0.8520 - val_loss: 0.4317 - learning_rate: 1.0000e-04
Epoch 7/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/ste

In [14]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = cnn_model_3.evaluate(test_dataset)
print(f'Test accuracy: {test_accuracy}')
print(f'Test loss: {test_loss}')

51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.9378 - loss: 0.1698
Test accuracy: 0.94772869348526
Test loss: 0.14866694808006287


# SA-CNN

## Trial 1: Without Attention Layer

In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define the Self-Attention Convolutional Neural Network (SA-CNN) model
def build_sacnn_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    # Convolutional Block 1
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)

    # Convolutional Block 2
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Convolutional Block 3
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Self-Attention Layer
    attention_output = layers.MultiHeadAttention(num_heads=4, key_dim=128)(x, x)
    x = layers.Add()([x, attention_output])  # Residual connection

    # Fully Connected Layer
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation="relu")(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Create model
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

# Initialize and compile the model
sa_cnn_model = build_sacnn_model(input_shape, num_classes)

In [16]:
# Compile the model
optimizer = optimizers.Adam(learning_rate=0.0001)
sa_cnn_model.compile(
    optimizer=optimizer, 
    loss="sparse_categorical_crossentropy", 
    metrics=["accuracy"]
)

# Define callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6)
# Train the model on the dataset
history = sa_cnn_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=[early_stopping, reduce_lr],
)

Epoch 1/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 92s 254ms/step - accuracy: 0.1613 - loss: 2.2019 - val_accuracy: 0.4025 - val_loss: 1.6807 - learning_rate: 1.0000e-04
Epoch 2/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 66s 211ms/step - accuracy: 0.3960 - loss: 1.6215 - val_accuracy: 0.4571 - val_loss: 1.5037 - learning_rate: 1.0000e-04
Epoch 3/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 66s 212ms/step - accuracy: 0.4785 - loss: 1.4242 - val_accuracy: 0.4508 - val_loss: 1.4865 - learning_rate: 1.0000e-04
Epoch 4/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 66s 212ms/step - accuracy: 0.5709 - loss: 1.2370 - val_accuracy: 0.6044 - val_loss: 1.1179 - learning_rate: 1.0000e-04
Epoch 5/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 66s 212ms/step - accuracy: 0.6226 - loss: 1.0507 - val_accuracy: 0.6652 - val_loss: 0.9511 - learning_rate: 1.0000e-04
Epoch 6/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 66s 212ms/step - accuracy: 0.6863 - loss: 0.8965 - val_accuracy: 0.6922 - val_loss: 0.8465 - learning_rate: 1.0000e-04
Epoch 7/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 66s 21

In [17]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = sa_cnn_model.evaluate(test_dataset)
print(f'Test accuracy: {test_accuracy}')
print(f'Test loss: {test_loss}')

51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 96ms/step - accuracy: 0.8626 - loss: 0.3843
Test accuracy: 0.873677670955658
Test loss: 0.36159056425094604


# PreTrained

## EfficientB0

In [18]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models, optimizers

def build_efficientnetb0_model(input_shape, num_classes):
    # Define input layer
    inputs = tf.keras.Input(shape=input_shape)

    # Load the EfficientNetB0 model with pre-trained ImageNet weights, excluding the top layer
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_tensor=inputs)
    base_model.trainable = False  # Freeze the base model layers

    # Add custom layers on top
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)  # Add global average pooling layer
    x = layers.Dropout(0.5)(x)  # Dropout layer to prevent overfitting
    x = layers.Dense(128, activation='relu')(x)  # Fully connected layer
    x = layers.Dropout(0.3)(x)  # Additional dropout layer

    # Output layer with softmax activation for multi-class classification
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = models.Model(inputs=inputs, outputs=outputs)

    return model

# Build the EfficientNetB0 model
efficientnetb0_model = build_efficientnetb0_model(input_shape, num_classes)

# Compile the model
optimizer = optimizers.Adam(learning_rate=0.0001)

efficientnetb0_model.compile(optimizer=optimizer, 
                             loss='sparse_categorical_crossentropy', 
                             metrics=['accuracy'])

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [19]:
# Define callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model on the dataset
history = efficientnetb0_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=[early_stopping, reduce_lr]  
)

Epoch 1/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 64s 132ms/step - accuracy: 0.0960 - loss: 2.3061 - val_accuracy: 0.1110 - val_loss: 2.2949 - learning_rate: 1.0000e-04
Epoch 2/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 16s 51ms/step - accuracy: 0.0953 - loss: 2.3064 - val_accuracy: 0.2006 - val_loss: 2.2954 - learning_rate: 1.0000e-04
Epoch 3/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 16s 50ms/step - accuracy: 0.0966 - loss: 2.3042 - val_accuracy: 0.1110 - val_loss: 2.3017 - learning_rate: 1.0000e-04
Epoch 4/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 15s 49ms/step - accuracy: 0.1017 - loss: 2.3036 - val_accuracy: 0.0596 - val_loss: 2.3020 - learning_rate: 1.0000e-04
Epoch 5/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 15s 49ms/step - accuracy: 0.0980 - loss: 2.3042 - val_accuracy: 0.0596 - val_loss: 2.3029 - learning_rate: 5.0000e-05
Epoch 6/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 15s 49ms/step - accuracy: 0.0914 - loss: 2.3035 - val_accuracy: 0.0596 - val_loss: 2.3028 - learning_rate: 5.0000e-05


In [20]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = efficientnetb0_model.evaluate(test_dataset)
print(f'Test accuracy: {test_accuracy}')
print(f'Test accuracy: {test_loss}')

51/51 ━━━━━━━━━━━━━━━━━━━━ 8s 161ms/step - accuracy: 0.0956 - loss: 2.2852
Test accuracy: 0.11076540499925613
Test accuracy: 2.29496169090271


## ResNet50

In [21]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models, optimizers

def build_resnet50_model(input_shape, num_classes):
    # Define input layer
    inputs = tf.keras.Input(shape=input_shape)

    # Load the ResNet50 model with pre-trained ImageNet weights, excluding the top layer
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)
    base_model.trainable = False  # Freeze the base model layers

    # Add custom layers on top
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)  # Add global average pooling layer
    x = layers.Dropout(0.5)(x)  # Dropout layer to prevent overfitting
    x = layers.Dense(128, activation='relu')(x)  # Fully connected layer
    x = layers.Dropout(0.3)(x)  # Additional dropout layer

    # Output layer with softmax activation for multi-class classification
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = models.Model(inputs=inputs, outputs=outputs)

    return model

# Build the model
resnet_model = build_resnet50_model(input_shape, num_classes)

# Compile the model
optimizer = optimizers.Adam(learning_rate=0.0001)

resnet_model.compile(optimizer=optimizer, 
                        loss='sparse_categorical_crossentropy', 
                        metrics=['accuracy'])

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [22]:
# Define callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model on the dataset
history = resnet_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=[early_stopping, reduce_lr]  
)

Epoch 1/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 67s 173ms/step - accuracy: 0.1059 - loss: 2.7530 - val_accuracy: 0.0627 - val_loss: 2.2920 - learning_rate: 1.0000e-04
Epoch 2/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 39s 126ms/step - accuracy: 0.1175 - loss: 2.3752 - val_accuracy: 0.1486 - val_loss: 2.3014 - learning_rate: 1.0000e-04
Epoch 3/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 39s 127ms/step - accuracy: 0.1104 - loss: 2.3226 - val_accuracy: 0.1129 - val_loss: 2.3021 - learning_rate: 1.0000e-04
Epoch 4/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 39s 126ms/step - accuracy: 0.1158 - loss: 2.3062 - val_accuracy: 0.1135 - val_loss: 2.3013 - learning_rate: 1.0000e-04
Epoch 5/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 40s 127ms/step - accuracy: 0.1208 - loss: 2.3013 - val_accuracy: 0.2658 - val_loss: 2.2940 - learning_rate: 5.0000e-05
Epoch 6/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 39s 126ms/step - accuracy: 0.1203 - loss: 2.2944 - val_accuracy: 0.2313 - val_loss: 2.2870 - learning_rate: 5.0000e-05
Epoch 7/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 40s 12

In [23]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = resnet_model.evaluate(test_dataset)
print(f'Test accuracy: {test_accuracy}')
print(f'Test accuracy: {test_loss}')

51/51 ━━━━━━━━━━━━━━━━━━━━ 8s 162ms/step - accuracy: 0.4556 - loss: 1.9347
Test accuracy: 0.25886744260787964
Test accuracy: 2.0360023975372314


## VGG-16

In [24]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers

def build_vgg16_model(input_shape, num_classes):
    # Define input layer
    inputs = tf.keras.Input(shape=input_shape)

    # Load the VGG16 model with pre-trained ImageNet weights, excluding the top layer
    base_model = VGG16(weights='imagenet', include_top=False, input_tensor=inputs)
    base_model.trainable = False  # Freeze the base model layers

    # Add custom layers on top
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)  # Add global average pooling layer
    x = layers.Dropout(0.5)(x)  # Dropout layer to prevent overfitting
    x = layers.Dense(128, activation='relu')(x)  # Fully connected layer
    x = layers.Dropout(0.3)(x)  # Additional dropout layer

    # Output layer with softmax activation for multi-class classification
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = models.Model(inputs=inputs, outputs=outputs)

    return model

# Build the VGG16 model
vgg16_model = build_vgg16_model(input_shape, num_classes)

# Compile the model
optimizer = optimizers.Adam(learning_rate=0.0001)

vgg16_model.compile(optimizer=optimizer, 
                    loss='sparse_categorical_crossentropy', 
                    metrics=['accuracy'])

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [25]:
# Define callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model on the dataset
history = vgg16_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=[early_stopping, reduce_lr]  
)

Epoch 1/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 133s 349ms/step - accuracy: 0.1041 - loss: 2.5081 - val_accuracy: 0.2433 - val_loss: 2.1999 - learning_rate: 1.0000e-04
Epoch 2/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 69s 222ms/step - accuracy: 0.1573 - loss: 2.2871 - val_accuracy: 0.4069 - val_loss: 2.1111 - learning_rate: 1.0000e-04
Epoch 3/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 69s 222ms/step - accuracy: 0.2228 - loss: 2.1783 - val_accuracy: 0.4840 - val_loss: 2.0211 - learning_rate: 1.0000e-04
Epoch 4/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 70s 223ms/step - accuracy: 0.2754 - loss: 2.0679 - val_accuracy: 0.5643 - val_loss: 1.9155 - learning_rate: 1.0000e-04
Epoch 5/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 70s 223ms/step - accuracy: 0.3161 - loss: 1.9746 - val_accuracy: 0.5712 - val_loss: 1.8151 - learning_rate: 1.0000e-04
Epoch 6/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 69s 223ms/step - accuracy: 0.3571 - loss: 1.8626 - val_accuracy: 0.6069 - val_loss: 1.6865 - learning_rate: 1.0000e-04
Epoch 7/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 70s 2

In [26]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = vgg16_model.evaluate(test_dataset)
print(f'Test accuracy: {test_accuracy}')
print(f'Test accuracy: {test_loss}')

51/51 ━━━━━━━━━━━━━━━━━━━━ 21s 412ms/step - accuracy: 0.7280 - loss: 0.9485
Test accuracy: 0.7510889768600464
Test accuracy: 0.8925820589065552
